In [ ]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder
from cosapp.drivers import NonLinearSolver, RunOnce, MonteCarlo, RunSingleCase
from cosapp.utils.distributions import Normal
import sys
sys.path.append('./model')
from Earth import Earth
import numpy as np

earth = Earth("earth")

l = 10 #Rocket's length on the plot
dt = 0.05 #Time-step


from cosapp.utils.distributions import Normal
from cosapp.drivers import MonteCarlo
from cosapp.recorders import DataFrameRecorder

In [ ]:

def run_analysis(syst, draws=10, linear=True):
    syst.drivers.clear()  # Remove all drivers on the System

    runonce = syst.add_driver(RunOnce("runonce"))
    driver = syst.add_driver(RungeKutta(order=4, dt=dt))
    driver.add_child(NonLinearSolver('solver', factor=1.0))
    driver.time_interval = (0, 40)

    # Define a simulation scenario
    driver.set_scenario(
        init = {
        'Traj.r' : np.array([0., 0., l/2]),
        'Rocket.Kin.v' : np.array([0,0,0]),
        'Rocket.Kin.ar' : np.array([0, -np.pi/2 + 0.2, 0]),
        'Rocket.Kin.av' : np.zeros(3),
        'Para.DynPar.r1' : np.array([0., 0., l/2]), #(should be l because the parachute is at the tip of the rocket)
        'Para.DynPar.r2' : np.array([0., 0., l/2]),
        'Para.DynPar.v1' : np.array([0,0,0]),
        'Para.DynPar.v2' : np.array([0,0,0]),
        'Traj.ParaDepStatus': False
    },
    stop='Para.DynPar.r1[2] < -1'
    )

    syst.run_drivers()
    print("MONTECARLO")
    # Montecarlo
    syst.drivers.clear()
    montecarlo = syst.add_driver(MonteCarlo('mc'))
    montecarlo.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r']))
    montecarlo.add_child(driver)
    montecarlo.draws = draws
    # montecarlo.linear = linear

    # parameters for uncertainties in the data
    cant_attr = syst.Rocket.Aero.Coefs.inwards.get_details('delta')
    # Set the distribution around the current value
    cant_attr.distribution = Normal(best=.05, worst=-0.05)
    montecarlo.add_random_variable('Rocket.Aero.Coefs.delta')

    # Computation
    syst.run_drivers()

    return montecarlo.recorder.export_data()


In [4]:

results = run_analysis(earth, draws=100, linear=False)

___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.422628923389915 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  164.17897833890765 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
MONTECARLO
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.422628923389915 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  164.1789783389076 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.422628923389915 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  164.17897833890765 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.422628923389915 m
Rocket Y Coordinate at Deployment:  -1.104619117061481e-15 m
Rocket Z Coordinate at Deployment:  164.1789783389076 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !


In [5]:
results

,Section,Status,Error code,Reference,Para.DynPar.r2,Para.DynPar.v2,Rocket.Thrust.inclinaison,initrot
0,,,0,0,"[43.29293369761231, -1.4010449059075962e-15, -...","[1.2145895256985987e-06, -4.075857176686508e-2...",-2.890607e-18,"[0.0, -1.4707963267948965, -1.0914015373614354..."
1,,,0,1,"[46.41420570065486, -8.543950871742066, -0.999...","[3.9789725966832856e-08, -1.7228733061455923e-...",2.603119e-01,"[0.0, -1.6009522551996178, 0.13015592840472132]"
2,,,0,2,"[-18.419098003853385, 2.7654962467230217, -1.0...","[6.356352588213091e-07, 1.7382946828005816e-06...",-2.603119e-01,"[0.0, -1.3406403983901751, -0.13015592840472132]"
3,,,0,3,"[94.05811730718278, -14.591803296986278, -0.99...","[4.835194378183283e-06, -2.295795116055674e-06...",1.229753e-01,"[0.0, -1.5322839954081209, -0.06148766861322437]"
4,,,0,4,"[16.008616156309373, -5.014150807099161, -1.00...","[1.5465016774527831e-06, 9.794870463288029e-07...",-4.439646e-01,"[0.0, -1.2488140186143666, 0.22198230818052986]"
...,...,...,...,...,...,...,...,...
95,,,0,95,"[-9.482079691601303, -2.5584601257265738, -1.0]","[-12.43340381815164, -3.354788077131791, -1.87...",5.686685e-01,"[0.0, -1.6856088344669293, -0.3399486022393575]"
96,,,0,96,"[-9.482079691601303, -2.5584601257265738, -1.0]","[-12.43340381815164, -3.354788077131791, -1.87...",-6.837673e-02,"[0.0, -1.4053195465254877, 0.018924931111407167]"
97,,,0,97,"[-9.482079691601303, -2.5584601257265738, -1.0]","[-12.43340381815164, -3.354788077131791, -1.87...",-3.537349e-01,"[0.0, -1.5283209908885695, 0.15498778285987097]"
98,,,0,98,"[-9.482079691601303, -2.5584601257265738, -1.0]","[-12.43340381815164, -3.354788077131791, -1.87...",1.801660e-01,"[0.0, -1.2413239145291852, -0.10731526005550862]"


In [6]:

results = run_analysis(earth, draws=10, linear=False)

In [ ]:
traj = np.asarray(results['Para.DynPar.r2'].tolist())
traj

array([[ 4.32929337e+01, -1.40104491e-15, -1.00000000e+00],
       [ 4.64142057e+01, -8.54395087e+00, -1.00000000e+00],
       [-1.84190980e+01,  2.76549625e+00, -1.00000000e+00],
       [ 9.40581173e+01, -1.45918033e+01, -1.00000000e+00],
       [ 1.60086162e+01, -5.01415081e+00, -1.00000000e+00],
       [ 1.03599360e+02,  1.98059188e+01, -1.00000000e+00],
       [ 1.79229076e-01,  4.83300136e-02, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00, -1.00000000e+00],
       [-9.48207969e+00, -2.55846013e+00

In [8]:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

traj = np.asarray(results['Traj.r'].tolist())
print(traj[:, 0])

# Create the figure object
fig = make_subplots(rows=1, cols=1)
fig.layout.title = "Probability"
fig.layout.yaxis.title = 'Y Position'

fig.add_trace(
    go.Scatter(
        x=traj[:,0],
        y=traj[:,1],
        mode = 'markers'
    ),
    row = 1,
    col = 1,
)
fig.get_subplot(1, 1).xaxis.title = "X Position"

fig.show()

In [9]:
results['Para.DynPar.r2']

0     [43.29293369761231, -1.4010449059075962e-15, -...
1     [46.41420570065486, -8.543950871742066, -0.999...
2     [-18.419098003853385, 2.7654962467230217, -1.0...
3     [94.05811730718278, -14.591803296986278, -0.99...
4     [16.008616156309373, -5.014150807099161, -1.00...
                            ...                        
95      [-9.482079691601303, -2.5584601257265738, -1.0]
96      [-9.482079691601303, -2.5584601257265738, -1.0]
97      [-9.482079691601303, -2.5584601257265738, -1.0]
98      [-9.482079691601303, -2.5584601257265738, -1.0]
99      [-9.482079691601303, -2.5584601257265738, -1.0]
Name: Para.DynPar.r2, Length: 100, dtype: object